# Predict Future Sales

Competition:
https://www.kaggle.com/c/competitive-data-science-predict-future-sales

Даны ежедневные исторические данные о продажах. Задача - спрогнозировать общее количество проданных товаров в каждом магазине по тестовой выборке. 

Описание файлов:

* sales_train.csv - тренировочный датасет. Ежедневыне продажи с января 2013 по октябрь 2015.
* test.csv - тестовый датасет. Необходимо предсказать продажи для этого набора товаров и магазинов на ноябрь 2015.
* sample_submission.csv - образец файла-решения.
* items.csv - дополнительная информация о товарах.
* item_categories.csv  - дополнительная информация о категориях товаров.
* shops.csv- дополнительная информация о магазинах.

## Импорт данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb

import warnings

%matplotlib inline
sns.set(style="darkgrid")
warnings.filterwarnings("ignore")

Загрузим данные

In [ ]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
item_cat = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
sales = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')

# отсавляем только те значения, которые есть в test, для экономии ресурсов
sales = sales.loc[sales["shop_id"].isin(test["shop_id"].unique()), :]
sales = sales.loc[sales["item_id"].isin(test["item_id"].unique()), :]

# Обзор датасетов

## Обзор вспомогательных датасетов

**Товары**

In [ ]:
items.shape

In [ ]:
items.head()

Где:
* item_name - текстовая номенклатура.
* item_id - уникальный код товара
* item_category_id - уникальный код категории товаров

**Категории товаров**

In [ ]:
item_cat.shape

In [ ]:
item_cat

Где:
* item_category_name - текстовое наименование категории.
* item_category_id - уникальный код категории товаров

Категория товара в большинстве случаев состоит из двух частей: категория и подкатегория. Это можно использовать для составления более точного прогноза.

In [ ]:
item_cat['cat'] = item_cat['item_category_name'].apply(lambda x: x.split(' - ')[0] if ('-' in x) else x)
item_cat['subcat'] = item_cat['item_category_name'].apply(lambda x: x.split(' - ')[1] if ('-' in x) else x)

# присваиваем каждой позиции числовую метку
item_cat['cat_code'] = LabelEncoder().fit_transform(item_cat['cat'])
item_cat['subcat_code'] = LabelEncoder().fit_transform(item_cat['subcat'])

item_cat.head(10)

**Магазины**

In [ ]:
shops.shape

In [ ]:
shops

Наименование магазина также состоит из двух частей: город и адрес, что тоже можно использовать для уточнения прогноза. 
Также при дальнейшей обработке данных следует учесть, что магазины "Якутск Орджоникидзе, 56", "Якутск ТЦ "Центральный"" и "Жуковский ул. Чкалова 39м2" имеют дубликаты, от которых нужно будет избавиться

In [ ]:
# Якутск Орджоникидзе, 56
sales.loc[sales['shop_id'] == 0, 'shop_id'] = 57
test.loc[test['shop_id'] == 0, 'shop_id'] = 57

# Якутск ТЦ "Центральный"
sales.loc[sales['shop_id'] == 1, 'shop_id'] = 58
test.loc[test['shop_id'] == 1, 'shop_id'] = 58

# Жуковский ул. Чкалова 39м2
sales.loc[sales['shop_id'] == 10, 'shop_id'] = 11
test.loc[test['shop_id'] == 10, 'shop_id'] = 11

Выделим город из адреса магазина, при этом нужно учесть, что есть такие магазины как "Выездная Торговля", "Интернет-магазин ЧС", "Цифровой склад 1С-Онлайн" их необходимо выделить отдельно, в категории "Выезд" и "Онлайн"

In [ ]:
# для корректного распределения городов, исправим наименование города, убрав пробел
shops.loc[shops['shop_name']=='Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'

In [ ]:
def city_select(adr):
    if adr == 'Выездная Торговля':
        return 'Выезд'
    if adr == 'Интернет-магазин ЧС' or adr == 'Цифровой склад 1С-Онлайн':
        return 'Онлайн'
    else:
        return adr.split()[0]

In [ ]:
shops['city'] = shops['shop_name'].apply(city_select)
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])

shops.head(15)

In [ ]:
test.shape

In [ ]:
test.head()

Дополним тестовые данные:

In [ ]:
# тестовые данные это прогнозируемый 34-й месяц
test['date_block_num'] = 34
test.drop(['ID'], axis=1, inplace=True)

In [ ]:
# добавим цены на товары
test = pd.merge(test, sales.groupby('item_id')['item_price'].last(), on='item_id', how='left')

In [ ]:
test.head()

In [ ]:
print('Уникальных пропусков: ', len(test[test['item_price'].isna()]['item_id'].unique()))
print('Всего пропусков: ', test['item_price'].isna().sum())

384 наименования товаров на тесте ранее не встречались. Необходимо заполнить пробелы. Для этого необходимо проставить среднюю цену в зависимости от группы товаров.

In [ ]:
mean_cat_df = pd.merge(sales, items[['item_id','item_category_id']], on='item_id', how='left')\
            .groupby('item_category_id')\
            .mean()\
            .reset_index()[['item_category_id','item_price']]

In [ ]:
fill_na_df = pd.merge(test[test['item_price'].isna()].reset_index(), \
                      items[['item_id','item_category_id']], on='item_id', how='left')

In [ ]:
test.loc[test['item_price'].isna(), 'item_price'] = pd.merge(fill_na_df, mean_cat_df, \
                                                             on='item_category_id', how='left').set_index('index')['item_price_y']

In [ ]:
print('Уникальных пропусков: ', len(test[test['item_price'].isna()]['item_id'].unique()))
print('Всего пропусков: ', test['item_price'].isna().sum())

Несколько новых товаров не волши ни в одну из категорий. Усредним их в целом по всему датасету:

In [ ]:
test['item_price'].fillna(test['item_price'].mean(), inplace=True)

In [ ]:
print('Уникальных пропусков: ', len(test[test['item_price'].isna()]['item_id'].unique()))
print('Всего пропусков: ', test['item_price'].isna().sum())

## Обзор датасета с данными о продажах

In [ ]:
sales.shape

In [ ]:
sales.head(10)

In [ ]:
sales.info()

Поле 'date' необходимо перевести в datetime

In [ ]:
sales['date'] = pd.to_datetime(sales['date'])

In [ ]:
sales['date'].head(10)

In [ ]:
sales[['item_price', 'item_cnt_day']].describe()

Пристутствуют отприцательные значения, что странно для полей с таким содержанием. Это скорее всего говорит о возвратах товара в случае с количеством продаж и об ошибке данных в случае цены товара.
Также стоит рассмотреть максимальные значения, они на порядок превосходят 75 квантиль, что скорее всего говорит о наличии выбросов.

In [ ]:
sales.isna().sum()

Пропусков в данных нет

## Обработка выбросов:

**В первую очередь рассмотрим выбросы**

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=sales['item_price'])

plt.figure(figsize=(10,4))
sns.boxplot(x=sales['item_cnt_day'])

На лицо явные выбросы в данных, очистим датасет от наиболее критичных значений и поближе рассмотрим товары с предополагаемыми выбросами.

In [ ]:
sales = sales[(sales['item_price'] < 50000) & (sales['item_cnt_day'] < 1000)]

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=sales['item_price'])

plt.figure(figsize=(10,4))
sns.boxplot(x=sales['item_cnt_day'])

Рассмотрим отрицательные значения в цене:

При анализе состава остальных данных, для простоты понимания добавим подписи для категорий и товаров по их ID

In [ ]:
sales_items = pd.merge(sales, items, on='item_id', how='left')
sales_items = pd.merge(sales_items, item_cat, on='item_category_id', how='left')

In [ ]:
sales_items.sort_values(by='item_price', ascending=False)[['item_price', 'item_cnt_day', 'item_name', 'item_category_name']].head(20)

Самыми дорогими позициями являются товары из категории Игровых консолей и Программного обеспечения. Они не являются выбросами, удалять такие данные нельзя

In [ ]:
sales_items.sort_values(by='item_cnt_day', ascending=False)[['item_price', 'item_name', 'item_category_name', 'item_cnt_day']].head(20)

Большинство самых объёмных покупок производилось через интернет магазины. Данные не являются выбросами, т.к. эти покупки могли совершаться корпоративными клиентами оптом.

# Экспресс-анализ данных

**Так как требуется помесячный прогноз определим для каждого товара суммарное количество продаж по каждому месяцу:**

In [ ]:
group = sales.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_price':['last'], 'item_cnt_day':['sum']})
group.reset_index(inplace=True)
group.columns = ['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_month']

In [ ]:
group.head()

Рассмотрим данные по-ближе, для этого создадим новый датасет, соержащий текстовое описание id-полей:

In [ ]:
comp_sales = pd.merge(group, items, on='item_id', how='left')
comp_sales = pd.merge(comp_sales, shops, on='shop_id', how='left')
comp_sales = pd.merge(comp_sales, item_cat, on='item_category_id', how='left')
comp_sales['value'] = comp_sales.item_price * comp_sales.item_cnt_month
comp_sales.head()

Рассмотрим линейную взаимосвзяь пар признаков:

In [ ]:
plt.figure(figsize=(10,8))

sns.heatmap(comp_sales.corr(), annot=True)

Заметна высокая корреляция между признаками "city_code" и 'shop_id', а также "item_category_id" и "cat_code", что не удивительно, ведь это производные одного показателя. 

In [ ]:
plt.figure(figsize=(25,8))

sns.barplot(data=comp_sales.groupby(by='city').sum().reset_index(), x="city", y='value')

plt.xticks(rotation=45)
plt.show()

In [ ]:
# количество торговых точек в городах
shops['city'].value_counts().head(15)

За рассматриваемый период лидером продаж является Москва, на втором месте Якутск, следом - Воронеж, что во многом обусловлено и количеством торговых точек в данных городах. Это говорит о класической модели ведения бизнеса: большинстов продаж осуществляется через торговые точки и офисы продаж, а не онлайн.

In [ ]:
plt.figure(figsize=(25,8))

sns.barplot(data=comp_sales.groupby(by='cat').sum().reset_index(), x="cat", y='value')

plt.xticks(rotation=45)
plt.show()

Самая продаваемая категория - это игры и игровые консоли.

Топ 20 самых продаваемых товаров:

In [ ]:
comp_sales.groupby(by='item_name').sum()['item_category_id'].sort_values(ascending=False).head(20)

Топ 20 самых неполпулярных товаров:

In [ ]:
comp_sales.groupby(by='item_name').sum()['item_category_id'].sort_values(ascending=False).tail(20)

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=1, figsize = (15,10))

online = comp_sales[comp_sales['city'] == 'Онлайн'].groupby(by='cat').sum()['value']

axes[0].set_title('Продажи "Онлайн"')
axes[0].pie(x=online, autopct="%.1f%%", labels=online.index, pctdistance=0.7)

offline = comp_sales[comp_sales['city'] != 'Онлайн'].groupby(by='cat').sum()['value']

axes[1].set_title('Продажи "Оффлайн"')
axes[1].pie(x=offline, autopct="%.1f%%", labels=offline.index, pctdistance=0.7)

plt.tight_layout()
plt.show()


В сфере онлайн покупок доминириуют Игры, Игровые консоли и Игры PC, похожая ситуация и с Оффлайн продажами, однако, доля продаж игровых консолей меньше, при этом весомую часть занимают Подарки, в Онлайн сегменте также можно выделить большую долю продаж Карт оплат и Программ, а также доставку. При этом доля оплаты за доставку превышает стоимость продаж большинства групп товаров.

In [ ]:
plt.figure(figsize=(25,8))
sns.barplot(data=comp_sales.groupby(by='subcat').sum().reset_index(), x="subcat", y='value')
plt.xticks(rotation=90)
plt.show()

Лидерами продаж выстуапают консоли от Sony и игры к ним.

**Рассмотрим динамику продаж по месяцам:**

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1,figsize=(25,8))

axes[0].set_title('Динамика выручки')
sns.lineplot(data=comp_sales.groupby(by='date_block_num').sum().reset_index(), x="date_block_num", y='value', ax=axes[0])

axes[1].set_title('Динамика количества проданных единиц(целевая переменная)')
sns.lineplot(data=comp_sales.groupby(by='date_block_num').sum().reset_index(), x="date_block_num", y='item_cnt_month', ax=axes[1])

plt.tight_layout()
plt.show()

Наблюдается восходящий тренд, а также явные пиковые точки продаж. Предположительно это связано с новгодними праздниками. Проверим:

In [ ]:
comp_sales.groupby(by='date_block_num').sum().sort_values(by='value', ascending=False)['value'].head(2)

Учитывая, что нумерация месяцев начинается с 0 и с 1-го января 2013 года, то 23 и 11 месяц это декабрь 2014 и декабрь 2013 соотвественно.

In [ ]:
plt.figure(figsize=(25,10))

sns.lineplot(data=comp_sales.groupby(by=['date_block_num', 'cat']).sum().reset_index(), x='date_block_num', y='item_cnt_month', hue='cat')

plt.title('Динамика продаж по категориям')

Наблюдается восходящий тренд на самые популярные товары, также наиболее яркие пики продаж заметны в декабре. Стоит обратить внимание и на периодические большие скачки продаж по играм, причиной могут быть сезонные распродажи от издателей или премьеры новых продуктов.

# Подготовка данных

In [ ]:
prp_sales = comp_sales[['shop_id', 'item_id', 'date_block_num', 'item_cnt_month', 'item_price']]
prp_sales.head()

Как мы видели выше, распределение продаж по точкам не однородно, следовательно вероятны ситуации, когда какой-то товар не продавался в магазине в определённом периоде. Для повышения качества прогноза, необходимо явно выделить такие ситуации, для этого необходимо расширить наш датасет и включить в него все возможные комбинации 'item_id' и 'shop_id' помесячно

In [ ]:
from itertools import product # функция возвращает все возможные комбинации итерируемых элементов

In [ ]:
inter_matrix = []

for month in prp_sales['date_block_num'].unique():
    shops_in_month = prp_sales.loc[prp_sales['date_block_num'] == month, 'shop_id'].unique()
    items_in_month = prp_sales.loc[prp_sales['date_block_num'] == month, 'item_id'].unique()
    inter_matrix.append(np.array(list(product(*[shops_in_month, items_in_month, [month]])), dtype='int32'))

In [ ]:
inter_matrix[:2]

In [ ]:
# стыкуем массивы в общий массив
inter_matrix = np.vstack(inter_matrix)
inter_matrix = pd.DataFrame(inter_matrix, columns=['shop_id', 'item_id', 'date_block_num'])
inter_matrix.shape

Добавим в полученную матрицу значения месячных продаж для каждой комбинации
Пропуски заполним нулями, и ограничим значения целевой переменной интервалом (0,20), чтобы приблизить к условиям задачи.

In [ ]:
prp_sales = pd.merge(inter_matrix, prp_sales, on=['shop_id', 'item_id', 'date_block_num'], how='left')
prp_sales['item_cnt_month'] = prp_sales['item_cnt_month'].fillna(0).clip(0,20)
prp_sales['item_price'] = prp_sales['item_price'].fillna(0)

In [ ]:
print("Пропущенных значений: ", prp_sales['item_cnt_month'].isna().sum())
print("Максимальное значение целевой переменной: ",prp_sales['item_cnt_month'].max())

Теперь необходимо добавить в итоговую матрицу основные метки связанные с товаром: магазины, категория, цена и т.д.

In [ ]:
prp_sales = pd.merge(prp_sales, items, on='item_id', how='left')
prp_sales = pd.merge(prp_sales, shops, on='shop_id', how='left')
prp_sales = pd.merge(prp_sales, item_cat, on='item_category_id', how='left')
prp_sales = prp_sales.drop(['item_name', 'shop_name', 'city', 'item_category_name', 'cat', 'subcat'], axis=1)
# добавим цену
prp_sales.head()

In [ ]:
prp_sales.shape

Заодно приведём к такому же виду набор данных для предсказаний:

In [ ]:
test = pd.merge(test, items[['item_id','item_category_id']], on='item_id', how='left')
test = pd.merge(test, item_cat[['item_category_id', 'cat_code', 'subcat_code']], on='item_category_id', how='left')
test = pd.merge(test, shops[['shop_id', 'city_code']], on='shop_id', how='left')
test['item_cnt_month'] = 0
test.head()

Теперь объденим тестовый дасет и датасет продаж:

In [ ]:
new_sales = prp_sales.append(test)

Для качественного построения моделей временного ряда необходимо учитывать  зависимость между настоящими и прошлыми значениями уровней данного ряда, это нужно для прогнозирования тренда или сезонности.

**Лагом** называется величина сдвига между рядами наблюдений.

Зададим временные лаги для ряда признаков. Будем сдвигать периоды помесячно до полугода и на год. Такой охват поможет оценить как трендовую зависимость так и сезонность.

In [ ]:
def generate_lag(data, months, lag_column):
    
    """Функция добавляет лаги в целевой признак. Принимает на вход датасет, величину сдвига и сам целевой признак"""
    
    for month in months:
        
        data_shift = data[['date_block_num', 'shop_id', 'item_id', lag_column]].copy()
        data_shift.columns = ['date_block_num', 'shop_id', 'item_id', lag_column + '_lag_' + str(month)]
        data_shift['date_block_num'] += month
        data = pd.merge(data, data_shift, on=['date_block_num', 'shop_id', 'item_id'], how='left')
    return data

**Целевой лаг:**

In [ ]:
%%time
new_sales = generate_lag(new_sales, [1,2,3,6,12], 'item_cnt_month')

**Сдвиг среднемесячных продаж по товарам:**

In [ ]:
group = new_sales.groupby(['date_block_num', 'item_id'])['item_cnt_month'].mean().rename('item_month_mean').reset_index()
new_sales = pd.merge(new_sales, group, on=['date_block_num', 'item_id'], how='left')
new_sales = generate_lag(new_sales, [1,2,3,6,12], 'item_month_mean')
new_sales.drop(['item_month_mean'], axis=1, inplace=True)

**Сдвиг среднемесячных продаж по магазинам:**

In [ ]:
group = new_sales.groupby(['date_block_num', 'shop_id'])['item_cnt_month'].mean().rename('shop_month_mean').reset_index()
new_sales = pd.merge(new_sales, group, on=['date_block_num', 'shop_id'], how='left')
new_sales = generate_lag(new_sales, [1,2,3,6,12], 'shop_month_mean')
new_sales.drop(['shop_month_mean'], axis=1, inplace=True)

**Сдвиг среднемесячных продаж по категориям:**

In [ ]:
group = new_sales.groupby(['date_block_num', 'cat_code'])['item_cnt_month'].mean().rename('cat_code_month_mean').reset_index()
new_sales = pd.merge(new_sales, group, on=['date_block_num', 'cat_code'], how='left')
new_sales = generate_lag(new_sales, [1,2,3,6,12], 'cat_code_month_mean')
new_sales.drop(['cat_code_month_mean'], axis=1, inplace=True)

**Сдвиг среднемесячных продаж по магазинам/категориям:**

In [ ]:
group = new_sales.groupby(['date_block_num', 'shop_id', 'item_category_id'])['item_cnt_month'].mean().rename('shop_category_month_mean').reset_index()
new_sales = pd.merge(new_sales, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
new_sales = generate_lag(new_sales, [1,2,3,6,12], 'shop_category_month_mean')
new_sales.drop(['shop_category_month_mean'], axis=1, inplace=True)

In [ ]:
new_sales.fillna(0, inplace=True)

In [ ]:
new_sales.shape

In [ ]:
new_sales.tail()

# XGBoost

В основе **XGBoost** лежит алгоритм градиентного бустинга деревьев решений. Градиентный бустинг — это техника машинного обучения для задач классификации и регрессии, которая строит модель предсказания в форме ансамбля слабых предсказывающих моделей, обычно деревьев решений. Считается одной из самых эффективных реализаций градиентного бустинга.

Перед использованием модели разделим нашу выборку на обучающую, валидационную и тестовую(в рамках этого задания - целевую). 

In [ ]:
x_train = new_sales[new_sales.date_block_num < 33].drop(['item_cnt_month'], axis=1)
y_train = new_sales[new_sales.date_block_num < 33]['item_cnt_month']

x_valid = new_sales[new_sales.date_block_num == 33].drop(['item_cnt_month'], axis=1)
y_valid = new_sales[new_sales.date_block_num == 33]['item_cnt_month']

x_test = new_sales[new_sales.date_block_num == 34].drop(['item_cnt_month'], axis=1)

Реализуем модель на основе XGBoost.

In [ ]:
%%time
model = xgb.XGBRegressor(
    n_estimators = 1000,
    learning_rate = 0.1,
    max_depth = 10,
    subsample = 0.5,
    colsample_bytree = 0.5)

model.fit(
    x_train, 
    y_train, 
    eval_metric='rmse', 
    eval_set=[(x_train, y_train),
               (x_valid, y_valid)],
    verbose=True,
    early_stopping_rounds=10,
          )

In [ ]:
# предсказания для оценочного набора данных
predictions = model.predict(x_test).clip(0,20)

# предсказание для валидационного набора данных
pred_val = model.predict(x_valid)

In [ ]:
model.best_score

Ошибка довольно низкая.

Визуализируем прогноз на валидационных данных

In [ ]:
plt.figure(figsize=(25,10))

sns.lineplot(x=x_valid.index, y=pred_val)
sns.lineplot(x=x_valid.index, y=y_valid, alpha=0.6)

**График прогноза довольно близко повторяет истинные значения. К тому же ошибка на валидации довольно низка - модель можно использовать для прогноза.**

Рассмотри оценку значимости признаков:

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,14))
xgb.plot_importance(model, ax=ax)

Признаки со сдвигами довольно сильно влияют на итоговый результат.

**Сохраним результат прогноза в файл:**

In [ ]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": predictions
})
submission.to_csv('submission.csv', index=False)